<a href="https://colab.research.google.com/github/jarmi95/03MAIR-Algoritmos-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Jarmi_Recio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jarmi Recio Martínez  <br>
Url: https://github.com/jarmi95/03MAIR-Algoritmos-Optimizacion/blob/main/Trabajo_Práctico_Algoritmos_Jarmi_Recio.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1q25a6YnfhPD4g2DXH5XkukFDtv3Ic-xN?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>
>2. <b>Organizar los horarios de partidos de La Liga</b><br>
>3. Configuración de Tribunales

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:

- Viernes: 20
- Sábado: 12, 16, 18, 20
- Domingo: 12, 16, 18, 20
- Lunes: 20

En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores (que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

|              | Categoría A  | Categoría B  | Categoría C |
|--------------|--------------|--------------|--------------|
| Categoría A  | 2 Millones   | 1,3 Millones | 1 Millón     |
| Categoría B  |              | 0,9 Millones | 0,75 Millones|
| Categoría C  |              |              | 0,47 Millones|


Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla.

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

|          | Viernes | Sábado  | Domingo | Lunes   |
|----------|---------|---------|---------|---------|
| 12h      |    -    |   0,55  |   0,45  |    -    |
| 16h      |    -    |   0,7   |   0,75  |    -    |
| 18h      |    -    |   0,8   |   0,85  |    -    |
| 20h      |   0,4   |    1    |    1    |   0,4   |


Es posible la coincidencia de horarios pero en este
caso la audiencia de cada partido se verá afectada y
se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de
coincidencias:

| Coincidencias | -% |
|---------------|----|
| 0             | 0%    |
| 1             | 25%   |
| 2             | 45%   |
| 3             | 60%   |
| 4             | 70%   |
| 5             | 75%   |
| 6             | 78%   |
| 7             | 80%   |
| 8             | 80%   |



                                        

#Modelo


## ¿Cómo represento el espacio de soluciones?


Se entiende por espacio de soluciones el conjunto de todas las posibles soluciones válidas para un problema dado.

En este caso, para la combinación de enfrentamientos entre equipos de La Liga de Fútbol dada, se consideran solución todas las combinaciones de horarios que siguen las restricciones enunciadas.

Dentro del espacio de soluciones se encuentran las dos principales variables de decisión: el equipo con su correspondiente categoría y el horario de enfrentamiento.

- Equipos: A1, A2, A3, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, C1, C2, C3, C4, C5 y C6.
- Horarios: V20, S12, S16, S18, S20, D12, D16, D18, D20 y L20.

La variable Equipo, pese a que se dan las combinaciones de la jornada, es relevante porque resulta directamente en una audiencia base u otra.

La variable Horario, además de implicar el factor reductor de la audiencia base por moemento en el que tiene lugar y por solapamiento, incluye la restricción de obligatoriedad en dos slots concretos.

Las múltiples combinaciones que se forman al asignar enfrentamientos en los diferentes horarios propuestos supone el total de soluciones.

Por ejemplo:

SOLUCIÓN 1: [B1 vs A1 - V20, B2 vs A2 - DS12, ... ]

SOLUCIÓN 2: [B1 vs A1 - S20, B2 vs A2 - D20, ... ]


In [ ]:
# A continuación se definen las variables del espacio de soluciones

Equipos = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']
CatA = ['A1', 'A2', 'A3']
CatB = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']
CatC = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']

Enfrentamientos = [
    ['B1','A1'],
    ['B2','A2'],
    ['C1','C2'],
    ['B3','A3'],
    ['C3','C4'],
    ['B4','C5'],
    ['B5','B6'],
    ['B7','B8'],
    ['B9','C6'],
    ['B10','B11']
]

Horarios = ['S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20']

Horarios_obligatorios = ['V20', 'L20']  # Horarios a asignar obligatoriamente


La siguiente función muestra la asignación aleatoria de horarios, generando un ejemplo de solución en cada ejecución.

Al añadirse la restricción de obligatoriedad de partidos en horario de lunes y viernes, cada ejecución podría considerarse como solución al problema.

Al no haberse implementado el cálculo de audiencias en función de las estadísticas aportadas en el enunciado, no se sabe cómo de buena o mala es cada una de las combinaciones generadas.

In [ ]:
import random

def horarios_aleatorios(enfrentamientos, horarios, horarios_obligatorios):
  enfrentamientos_con_horarios = []

  # Sublistas a las que se asignarán horarios obligatorios de la lista_adicional
  sublistas_obligatorias = random.sample(range(len(enfrentamientos)), len(horarios_obligatorios))
  # Verificar si el enfrentamiento es una de las sublistas con los horarios obligatorios
  for i, enfrentamiento in enumerate(enfrentamientos):
    if i in sublistas_obligatorias:
      # Asignar horario obligatorio
      horario = horarios_obligatorios[sublistas_obligatorias.index(i)]
    else:
      # Seleccionar aleatoriamente un horario no utilizado previamente en otros enfrentamientos
      horario = random.choice([h for h in horarios if h not in enfrentamientos_con_horarios])

    # Agregar el enfrentamiento con horario a la lista resultante
    enfrentamiento_con_horario = enfrentamiento + [horario]
    enfrentamientos_con_horarios.append(enfrentamiento_con_horario)

  return enfrentamientos_con_horarios

In [ ]:
enfrentamientos_con_horarios = horarios_aleatorios(Enfrentamientos, Horarios, Horarios_obligatorios)

# Imprimir la solución
for i in enfrentamientos_con_horarios:
  print(i)

['B1', 'A1', 'D20']
['B2', 'A2', 'S18']
['C1', 'C2', 'S18']
['B3', 'A3', 'S16']
['C3', 'C4', 'S20']
['B4', 'C5', 'S20']
['B5', 'B6', 'V20']
['B7', 'B8', 'S18']
['B9', 'C6', 'L20']
['B10', 'B11', 'S12']


## ¿Cuál es la función objetivo?

Una vez conocido el espacio de posibles soluciones, es necesario ajustarlo al objetivo del problema, es decir, maximizar la audiencia de la jornada de La Liga.

Para optimizar la combinación de horarios que devuelva la mayor audiencia posible en la jornada es necesario tener en cuenta tres factores:
- Estadísticas asociadas a categoría (F_Cat): las audiencias son mayores para enfrentamientos entre equipos de mayor categoría. Dado que los enfrentamientos son un dato conocido, el cálculo de este valor puede compararse con el de la audiencia base sobre el que se aplicarán factores de reducción.
- Estadísticas asociadas al horario (F_hor): las audiendias son máximas, en coeficiente, sábado y domingo a las 20h.
- Estadísticas de coincidencia (F_Coin): las coincidencias entre dos o más partidos en una franja horaria supone una reducción de la audiencia.

Por tanto, la función que recoge el valor de cada enfrentamiento se calcula con la siguiente ecuación:

Func_partido_i = F_Cat * F_Hor * F_Coin

Y la función total a optimizar será la suma de las audiencias de cada partido:

Func_tot = sum(Func_paprtido_1 + Func_partido_2 + ... + Func_partido_10)

### Audiancia base y factores

De cara a una mejor comprensión de la función a optimizar, se codifica tanto el cálculo de la audiencia base como los factores reductores de audiencia en función de los horarios y el solapamiento.


La siguiente función calcula la audicienda base de una jornada teniendo en cuenta:
- El factor de la categorías de los equipos que se enfrentan
- El factor del horario asignado

*Es necesario añadir que para calcular la función objetivo real, aún se tiene que incluir la penalización por solapamiento de horarios.

In [ ]:
def aud_base(jornada):

  jornada_corregida = []

  for partido in jornada:
    local = partido[0]
    visitante = partido[1]
    slot = partido[2]

    # Se ifentifican los equipos y su audiencia por petenencia a categoría
    if local in CatA and visitante in CatA:
      aud_base = 2
    elif local in CatB and visitante in CatB:
      aud_base = 0.9
    elif local in CatC and visitante in CatC:
      aud_base = 0.47
    elif (local in CatA and visitante in CatB) or (local in CatB and visitante in CatA):
      aud_base = 1.3
    elif (local in CatA and visitante in CatC) or (local in CatC and visitante in CatA):
      aud_base = 1
    elif (local in CatB and visitante in CatC) or (local in CatC and visitante in CatB):
      aud_base = 0.75

    # Se aplica la corrección por horario
    horarios = {'S20': 1, 'D20': 1, 'V20': 0.4, 'L20': 0.4, 'S12': 0.55, 'S16': 0.7, 'S18': 0.8, 'D12': 0.45, 'D16': 0.75, 'D18': 0.85}
    aud_base_slot = aud_base * horarios.get(slot, 0)

    partido_corregido = partido + [aud_base_slot]
    jornada_corregida.append(partido_corregido)

  return jornada_corregida

In [ ]:
# Asignación aleatoria de horario
enfrentamientos_con_horarios = horarios_aleatorios(Enfrentamientos, Horarios, Horarios_obligatorios)

# Cálculo de audiendias sin corrección por solapamiento
audiencia_base = aud_base(enfrentamientos_con_horarios)
for i in audiencia_base:
  print(i)

['B1', 'A1', 'D12', 0.5850000000000001]
['B2', 'A2', 'D16', 0.9750000000000001]
['C1', 'C2', 'S12', 0.2585]
['B3', 'A3', 'S16', 0.9099999999999999]
['C3', 'C4', 'D12', 0.2115]
['B4', 'C5', 'S20', 0.75]
['B5', 'B6', 'V20', 0.36000000000000004]
['B7', 'B8', 'S16', 0.63]
['B9', 'C6', 'D16', 0.5625]
['B10', 'B11', 'L20', 0.36000000000000004]


Tras el cálculo de audiencias base con el correspondiente factor horario, se aplican las reducciones de audiencia en base a los solapamientos para obtener valores reales de la función a optimizar.

La siguiente función incluye los factores de reducción por solapamiento:

In [ ]:
def aud_finales(audiencias_jornada):

  # Se crea un contador para contabilizar las repeticiones en horario
  contador = {}
  for sublista in audiencias_jornada:
    primera_posicion = sublista[-2]
    if primera_posicion in contador:
      contador[primera_posicion] += 1
    else:
     contador[primera_posicion] = 1
  #print("\nHorarios asignados: \n",contador)

  # Se aplica el factor de solapamiento para cada fecha con más de un enfrentamiento
  for clave, valor in contador.items():
    if valor > 1:
      indices_coincidentes = [i for i, sublista in enumerate(audiencias_jornada) if sublista[-2] == clave]
      # Multiplicar por el factor
      for indice in indices_coincidentes:
        if valor == 2:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.75
        elif valor == 3:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.55
        elif valor == 4:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.4
        elif valor == 5:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.3
        elif valor == 6:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.25
        elif valor == 7:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.22
        elif valor == 8:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.2
        elif valor == 9:
          audiencias_jornada[indice][-1] = audiencias_jornada[indice][-1] * 0.2

  return audiencias_jornada

El siguiente código permite comparar los resultados de audiencia obtenidos sin y con la corrección de solapamiento:

In [ ]:
# Asignación aleatoria de horario
enfrentamientos_con_horarios = horarios_aleatorios(Enfrentamientos, Horarios, Horarios_obligatorios)

print('\nAudiencias sin corrección de solapamiento:')
# Cálculo de audiendias sin corrección por solapamiento
audiencia_base = aud_base(enfrentamientos_con_horarios)
for i in audiencia_base:
  print(i)
print(f"Audiencia total: {sum(sublista[-1] for sublista in audiencia_base if sublista)}")

# Se aplica la función de solapamiento
audiendia_con_solapamiento = aud_finales(audiencia_base)
print('\nAudiencias con corrección de solapamiento:')
for i in audiendia_con_solapamiento:
  print(i)
print(f"Audiencia total: {sum(sublista[-1] for sublista in audiendia_con_solapamiento if sublista)}")


Audiencias sin corrección de solapamiento:
['B1', 'A1', 'L20', 0.52]
['B2', 'A2', 'D20', 1.3]
['C1', 'C2', 'D20', 0.47]
['B3', 'A3', 'S16', 0.9099999999999999]
['C3', 'C4', 'D20', 0.47]
['B4', 'C5', 'D20', 0.75]
['B5', 'B6', 'S12', 0.49500000000000005]
['B7', 'B8', 'D18', 0.765]
['B9', 'C6', 'V20', 0.30000000000000004]
['B10', 'B11', 'D20', 0.9]
Audiencia total: 6.88

Audiencias con corrección de solapamiento:
['B1', 'A1', 'L20', 0.52]
['B2', 'A2', 'D20', 0.39]
['C1', 'C2', 'D20', 0.141]
['B3', 'A3', 'S16', 0.9099999999999999]
['C3', 'C4', 'D20', 0.141]
['B4', 'C5', 'D20', 0.22499999999999998]
['B5', 'B6', 'S12', 0.49500000000000005]
['B7', 'B8', 'D18', 0.765]
['B9', 'C6', 'V20', 0.30000000000000004]
['B10', 'B11', 'D20', 0.27]
Audiencia total: 4.157


## ¿Cómo implemento las restricciones?

La principal restricción a incluir, además de la reducción de las audiencias base en función de los coeficientes comentados anteriormente, es la obligatoriedad de un partido en horario de viernes y lunes pese a tener el coeficiente más bajo.

Esta condición implica que, dentro de los 10 posibles horarios a elegir para cada enfrentamiento, habrá 2 que siempre deberán estar presentes en la solución de la jornada óptima. Dejando los 8 horarios restantes como alternativas "flexibles" para el resto de partidos.

Tendiendo a las implicaciones de los horarios de Lunes y Viernes, se observa una reducción del 60% de la audiciencia. Como esta cifra es muy superior a cualquier otro horario, se tenderá a la no repetición de partidos los Lunes y Viernes.

Esta restricción está incluida en la función 'horarios_aleatorios' del primer apartado, donde se asignan obligatoriamente esos horarios para dos enfrentamiento cualquiera de la jornada.


#Análisis


### ¿Qué complejidad tiene el problema?


####Contabilizar el espacio de soluciones

Para contabilizar el espacio de soluciones se parte de analisis realizado en el apartado anterior.

Teniendo en cuenta que los enfrentamientos son un parámetro dado y que se tienen que asignar 10 horarios de forma que se maximice la audiencia, el número de combinaciones posible es **10^10, es decir, 10 mil millones de soluciones posibles**.

Sobre esta cifra tiene que hacerse el ajuste asociado a la restricción de los partidos de lunes y viernes. Ya que dos de los partidos tendrán que jugarse en esos horarios obligatoriamente, la combinación de soluciones se reduce a: **8 * 9 * 10^8**.

#### Orden de complejidad

El orden de complejidad se relaciona con la complejidad empleada por el algoritmo para resolver el problema en cuestión.

En el siguiente aparetado se proponen dos técnicas para resolver el problema de optimización de la jornada de La Liga:

1. Búsqueda aleatoria

Esta técnica parte del código desarrollado en el apartado anterior para detallar las fases de modelado.

Atendiendo al código, se observa la llamada de 3 funciones principales, cada una de estas con la siguiente complejidad de código:

- horarios_aleatorios(): O(n)
- aud_base(): O(n)
- aud_finales(): O(n)

Adicionalmente, en la función que invoca a las tres funciones mencionadas, se incluye un bucle para la colección de audiencias, que equivaldría a una complejidad de O(n).

Dado que el código se ejecuta de forma secuancial sin incurrir en bucles anidados, **la complejidad total del código es de O(n)**, pero, para realizar una búsquqeda sobre N soluciones, es necesario introducir la función general en un bluque principal, **incrementando a O(n^2) la complejidad**.

2. Búsqueda Local

Para la técnica de búsqueda local se invocan a las siguientes funciones, con los siguientes valores de complejidad:

- audiencias_final_jornada(): O(n)
- genera_vecina(): O(n)
- aud_base(): O(n)
- aud_finales(): O(n)

Además del bucle comentado anteriormente para la obtención de las audiencias finales.

Dado que las tres últimas funciones comentadas anteriormente se integran en un bucle principal, **la complejidad final del desarrollo es O(n^2)**.

#Diseño

## ¿Qué técnica utilizo y por qué?



## Búsqueda aleatoria

Este algoritmo heurístico, pese a parecer algo ingenuo, puede resultar útil en problemas complejos con restricciones y para los que no existe un problema determinista fácil de desarrollar.

Además de lo comentado, se elige esta técnica como una opción sencilla de implementar y capaz de explorar de forma diversa el gran espacio de soluciones que presenta el problema.

En su contra, además de que no se garantiza la solución óptima, es necesario reconocer que a priori no será de las mejores técnicas en cuanto a eficiencia y tiempo de ejecución.

La siguiente función recoge el proceso utilizado en el apartado de modelado:

In [ ]:
def audiencias_final_jornada(Enfrentamientos, Horarios, Horarios_obligatorios):

  # Asignación aleatoria de horario
  enfrentamientos_con_horarios = horarios_aleatorios(Enfrentamientos, Horarios, Horarios_obligatorios)

  # Cálculo de audiendias sin corrección por solapamiento
  audiencia_base = aud_base(enfrentamientos_con_horarios)

  # Se aplica la función de solapamiento
  audiendia_con_solapamiento = aud_finales(audiencia_base)
  aud_tot = sum(sublista[-1] for sublista in audiendia_con_solapamiento if sublista)

  return audiendia_con_solapamiento, aud_tot

jornada, audiencia = audiencias_final_jornada(Enfrentamientos, Horarios, Horarios_obligatorios)

print('\nAudiencias con corrección de solapamiento:')
for i in audiendia_con_solapamiento:
  print(i)
print(f"Audiencia total: {audiencia}")


Audiencias con corrección de solapamiento:
['B1', 'A1', 'L20', 0.52]
['B2', 'A2', 'D20', 0.39]
['C1', 'C2', 'D20', 0.141]
['B3', 'A3', 'S16', 0.9099999999999999]
['C3', 'C4', 'D20', 0.141]
['B4', 'C5', 'D20', 0.22499999999999998]
['B5', 'B6', 'S12', 0.49500000000000005]
['B7', 'B8', 'D18', 0.765]
['B9', 'C6', 'V20', 0.30000000000000004]
['B10', 'B11', 'D20', 0.27]
Audiencia total: 4.385325


Se introduce la generación aleatoria de soluciones en un bucle con capacidad de detención en base a un máximo de iteraciones y se almacena el mejor resultado obtenido:

MEJOR RESULTADO OBTENIDO: 6.85.

In [ ]:
from tqdm import tqdm

# Valor inicial para comparar
mayor_audiencia = float('-inf')
contador = 0
max_iter = 5000
max_sin_mejora = 1000

# Bucle para ejecutar la función N veces y actualizar el mayor resultado
for i in tqdm(range(max_iter), desc="Processing", unit="iteration"):
    # Llamada a la función para obtener la jornada y la audiencia
    jornada, audiencia = audiencias_final_jornada(Enfrentamientos, Horarios, Horarios_obligatorios)

    # Actualizar el mayor resultado
    if audiencia > mayor_audiencia:
        mayor_audiencia = audiencia
        jornada_top = jornada
        contador = 0  # Restablecer el contador si hay una mejora
        iter = i
    else:
        contador += 1

    # Verificar si ha habido X ejecuciones consecutivas sin mejorar
    if contador >= max_sin_mejora:
      break


print(f"\nLa mejor audiencia obtenida es de: {mayor_audiencia} en la iteración {i}")
for i in jornada_top:
  print(i)

Processing:  26%|██▌       | 1281/5000 [00:00<00:00, 9467.09iteration/s] 


La mejor audiencia obtenida es de: 6.718500000000001 en la iteración 1281
['B1', 'A1', 'D18', 1.105]
['B2', 'A2', 'D16', 0.9750000000000001]
['C1', 'C2', 'V20', 0.188]
['B3', 'A3', 'D20', 1.3]
['C3', 'C4', 'L20', 0.188]
['B4', 'C5', 'S18', 0.6000000000000001]
['B5', 'B6', 'S12', 0.49500000000000005]
['B7', 'B8', 'S20', 0.675]
['B9', 'C6', 'S20', 0.5625]
['B10', 'B11', 'S16', 0.63]


## Búsqueda Local

Se propone la aplicación del algoritmo de búsqueda local para poner el foco en las soluciones que implican vecindad y poca variabilidad respecto a una solución inicial.

Este enfoque contrasta con el del primer algoritmo implementado, búsqueda aleatoria, ya que esta tenía un componente de diversificación con mayor peso.

Por tanto, lo que se busca con este algoritmo es la exploración en regiones próximas mediante cambios controlados en la solución incial.

En primer lugar se codifica la función que genera las variaciones en la vecinidad que permiten la búsqueda local en base a la jornada de referencia.

Esta función implementa dos formas de búsqueda local mediante los parámetros "modo" y "num_intercambios":
- modo: puede tomar los valores de random o swap.
  - modo random: se realiza una aleatorización de las horas definidas en la jornada de referencia para generar nuevas combinaciones en base a los horarios existentes.
  - modo swap: re realiza el intercambio de hora entre pares de enfrentamientos.
- num_intercambios: número de pares que se intercambian la hora en el modo swap.

In [ ]:
def genera_vecina(jornada_referencia, modo, num_intercambios):
    # Clonar la jornada de referencia
    jornada_vecina = [sublista.copy() for sublista in jornada_referencia]

    if modo == "random":
        # Intercambiar aleatoriamente los horarios
        for sublista in jornada_vecina:
            indice_nuevo = random.randint(0, len(jornada_vecina) - 1)
            sublista[2], jornada_vecina[indice_nuevo][2] = jornada_vecina[indice_nuevo][2], sublista[2]

    elif modo == "swap":
        # Intercambiar los horarios entre dos enfrentamientos
        for _ in range(num_intercambios):
            indices_aleatorios = random.sample(range(len(jornada_vecina)), 2)
            jornada_vecina[indices_aleatorios[0]][2], jornada_vecina[indices_aleatorios[1]][2] = (
                jornada_vecina[indices_aleatorios[1]][2],
                jornada_vecina[indices_aleatorios[0]][2],
            )

    return jornada_vecina

A continuación, se desarrolla la función principal de búsqueda local que invoca a la generaciones de soluciones vecinas tras la inicialización de una jornada de referencia.

MEJOR RESULTADO OBTENIDO:

- random: 7.51 en < 40sg (Sim bargo se tienen resultados de más duración en los que la audiencia es mucho menor, alrededor de 5. Esto indica que la elección inicial de los horarios condiciona enormemente la solución final dada por la búsqueda local, como era previsible.)

- swap (4 pares): 7.64 < 1sg

In [ ]:
def busqueda_local(Enfrentamientos, Horarios, Horarios_obligatorios):

  contador = 0
  max_iter = 1000000
  max_sin_mejora = 1000

  # Generar una solucion inicial de referencia aleatoria con la funció previamente desarrollada
  jornada_referencia, audiencia_referencia = audiencias_final_jornada(Enfrentamientos, Horarios, Horarios_obligatorios)
  # Se define la mejor audiencia en base a la referencia
  mayor_audiencia = audiencia_referencia

  for i in tqdm(range(max_iter), desc="Processing", unit="iteration"):

    # Generación de una solución vecina
    jornada_vecina = genera_vecina(jornada_referencia, 'swap', 4)

    # Evaluación de audiencia (se usan las funciones desarrolladas anteriormente)
    audiencia_base_vecina = aud_base(jornada_vecina)
    # Se aplica la función de solapamiento
    audiendia_con_solapamiento_vecina = aud_finales(audiencia_base_vecina)
    # Se calcula la audiencia total vecina
    aud_vecina = sum(sublista[-1] for sublista in audiendia_con_solapamiento_vecina if sublista)

    # Actualizar el mayor resultado
    if aud_vecina > mayor_audiencia:
        mayor_audiencia = aud_vecina
        jornada_top = audiendia_con_solapamiento_vecina
        contador = 0  # Restablecer el contador si hay una mejora
        iter = i
    else:
        contador += 1

    # Verificar si ha habido X ejecuciones consecutivas sin mejorar
    if contador >= max_sin_mejora:
      break

  print(f"\nLa audiencia referencia es de: {audiencia_referencia}")
  for i in jornada_referencia:
    print(i)

  print(f"\nLa mejor audiencia obtenida es de: {mayor_audiencia}")
  for i in jornada_top:
    print(i)

busqueda_local(Enfrentamientos, Horarios, Horarios_obligatorios)

Processing:   0%|          | 2631/1000000 [00:00<00:35, 27780.62iteration/s]


La audiencia referencia es de: 6.200500000000001
['B1', 'A1', 'D18', 1.105]
['B2', 'A2', 'S20', 0.9750000000000001]
['C1', 'C2', 'V20', 0.188]
['B3', 'A3', 'S12', 0.7150000000000001]
['C3', 'C4', 'D16', 0.3525]
['B4', 'C5', 'L20', 0.30000000000000004]
['B5', 'B6', 'S18', 0.54]
['B7', 'B8', 'S20', 0.675]
['B9', 'C6', 'S18', 0.45000000000000007]
['B10', 'B11', 'D20', 0.9]

La mejor audiencia obtenida es de: 7.3885000000000005
['B1', 'A1', 'D18', 1.105, 1.105]
['B2', 'A2', 'S20', 0.9750000000000001, 1.3]
['C1', 'C2', 'V20', 0.188, 0.188]
['B3', 'A3', 'D20', 0.7150000000000001, 1.3]
['C3', 'C4', 'L20', 0.3525, 0.188]
['B4', 'C5', 'S12', 0.30000000000000004, 0.41250000000000003]
['B5', 'B6', 'S18', 0.54, 0.7200000000000001]
['B7', 'B8', 'D16', 0.675, 0.675]
['B9', 'C6', 'S18', 0.45000000000000007, 0.6000000000000001]
['B10', 'B11', 'S20', 0.9, 0.9]


# Próximos pasos

El siguiente apartado consiste en la resolución del problema empleando algoritmos genéticos.

**Se plantea como trabajo futuro debido a la complejidad de su código y a ciertos errores encontrados en la forma de resolverlo que extienden su desarrollo en el tiempo. Se han planteado las diversas funciones que componen el bucle final pero está inacabado. Necesita de resolver algunos aspectos de código que colisionan debido al uso de funciones desarrolladas para los otros algoritmos.**



## Algoritmos genéticos

Teniendo en cuenta que de las pruebas anteriores, la búsqueda local conseguía mejores resultados con un menor consumo de recursos gracias a una buena elección inicial. Se confía en un algoritmo genético que parta de una población aleatoria e itere seleccionando los mejores individuos de esta para perpetuar una especia de soluciones óptimas.

En caso de que este algoritmo genético se quisiera orientar como un algoritmo evolutivo, se pueden observar dos "problemas" que aparecerían una vez seleccionada la mejor población inicial dada la restricción de tener partidos los Viernes y los Lunes obligatoriamente:

- Los cruces deberían preservar la aparición de estos horarios en los descendientes.
- Las mutaciones deberían alterar los horarios de forma que tanto Lunes como Viernes sigan siendo parte del individuo mutado.

Ambas técnicas parecen más propias de un laboratorio que producto del propio proceso de evolución humana, al menos de la evolución definida por Darwin.

Por esto, se decide un algoritmo genético donde se puedan ejecutar estas técnicas de laboratorio sin ofender a ningún científico.

En primer lugar, se desarrolla la función de población inicial, basada en la generación de n_progenitores iniciales.

Como la función inicial también genera la audiencia asociada a cada solución, se realiza una ordenación sobre estas y se devuelven las n_mejores.

In [ ]:
def Generar_Poblacion(Enfrentamientos, Horarios, Horarios_obligatorios,n_progenitores, n_mejores):
  poblacion = []
  mejores_resultados = []
  max_aud_poblacion = float('-inf')
  mejor_pob = None

  # Generación de población inicial aleatoria
  for i in range(n_progenitores):
    jornada_referencia, audiencia_referencia = audiencias_final_jornada(Enfrentamientos, Horarios, Horarios_obligatorios)
    poblacion.append([jornada_referencia,audiencia_referencia])

    # Almacenar jornada de máxima audiencia
    if audiencia_referencia > max_aud_poblacion:
      mejor_pob = jornada_referencia
      max_aud_poblacion = audiencia_referencia

  # Ordenar descendente la lista en base a la audiencia calculada anteriormente
  mejores = sorted(poblacion, key=lambda x: x[1], reverse=True)

  return mejores[:n_mejores], mejor_pob, max_aud_poblacion

Tras seleccionar los mejores individuos de la población aleatoria, se procede al cruce de estos y a una mutación "selectiva".

In [ ]:
def Cruce_Mutacion(mejores):

  hijo = []

  # Restricciones
  slots_obligatorios = ['L20', 'V20']
  alternativas = ['S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20']

  if len(mejores) % 2 != 0:
    print("Error: Número impar de padres.")
    return 0

  print(mejores)

  for i in range(0, len(mejores), 2):
    pareja = mejores[i:i+2]

    # Cruce de mejores soluciones por la mitad
    m1, m2 = pareja[0][0], pareja[1][0]
    cruce = m1[:5] + m2[5:]

    # Variables de confirmación de la presencia de los horarios obligatorios
    l20 = any('L20' in sublist for sublist in cruce)
    v20 = any('V20' in sublist for sublist in cruce)
    # Verificar la presencia
    if l20 and v20:
      # Se contabiliza si hay solapamiento en los partidos de Lunes/Viernes
      l20_count = sum('L20' in sublist for sublist in cruce)
      v20_count = sum('V20' in sublist for sublist in cruce)
      # Se reemplazan los excesos de partidos en horario de Lunes/Viernes
      if l20_count > 1:
        for sublist in cruce:
          if 'L20' in sublist:
            sublist[2] = random.choice(alternativas)
      if v20_count > 1:
        for sublist in cruce:
          if 'V20' in sublist:
            sublist[2] = random.choice(alternativas)
      hijo.append(cruce)

    # Realizar mutaciones para forzar la aparición de Lunes/Viernes
    elif not l20 and not v20:
      cruce[2] = 'L20'
      cruce[2] = 'V20'
      hijo.append(cruce)
    elif not l20:
      random_index = random.choice(range(len(cruce[0])))
      if cruce[random_index][2] != 'V20':
        cruce[random_index][2] = 'L20'
      else:
        cruce[random_index - 1][2] = 'L20'
      hijo.append(cruce)
    elif not v20:
      random_index = random.choice(range(len(cruce[0])))
      if cruce[random_index][2] != 'L20':
        cruce[random_index][2] = 'V20'
      else:
        cruce[random_index - 1][2] = 'V20'
      hijo.append(cruce)
  # Se eliminan las audiencias pasadas de las sub-listas
  return [[[subelem for j, subelem in enumerate(sublist) if j != 3] for sublist in sublist_list] for sublist_list in hijo]

Para evaluar la población tras cuces/mutaciones se usan las funciones desarrolladas anteriormente que permiten calcular la audiencia final a partir de una jornada.

In [ ]:
def Eval_Selec(poblacion):

  mejores_resultados = []
  max_aud_poblacion = float('-inf')
  mejor_pob = None

  for i in poblacion:
    # Evaluación de audiencia (se usan las funciones desarrolladas anteriormente)
    audiencia_base_cruce = aud_base(poblacion)
    # Se aplica la función de solapamiento
    audiendia_con_solapamiento_cruce = aud_finales(audiencia_base_cruce)
    # Se calcula la audiencia total vecina
    aud_poblacion = sum(sublista[-1] for sublista in audiendia_con_solapamiento_cruce if sublista)

    if aud_poblacion > max_aud_poblacion:
      max_aud_poblacion = aud_poblacion
      mejor_pob = i

  mejores_resultados.append([i, aud_poblacion])

  return mejores_resultados, max_aud_poblacion, mejor_pob

Finalmente se genera la función que invoca a la generación de población, la selección y el cruce de los mejores.

In [ ]:
def algoritmo_genetico(Enfrentamientos, Horarios, Horarios_obligatorios,n_progenitores=1000, n_mejores=100, generaciones=10):
  # Se genera la mejor población inicial
  poblacion, mejor_pob, max_aud_poblacion = Generar_Poblacion(Enfrentamientos, Horarios, Horarios_obligatorios,n_progenitores, n_mejores)

  print(f'\nMejor individuo: con audiencia de: {max_aud_poblacion}\n')
  print(mejor_pob)

  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    # Cruce de la poblacion(incluye mutación)
    poblacion = Cruce_Mutacion(poblacion)

    # Evaluación de la mejor población y selección de la audiencia
    resultados_cruce, max_aud_cruce, mejor_cruce = Eval_Selec(poblacion)

    if max_aud_cruce > max_aud_poblacion:
      mejor_pob = mejor_cruce
      max_aud_poblacion = max_aud_cruce

    print(f'\nGeneración: #{n}. Mejor individuo: {mejor_pob} con audiencia de: {max_aud_poblacion}\n')

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_pob, max_aud_poblacion